In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.datasets import cifar100
from keras import applications
from __future__ import print_function
from keras import optimizers
from keras.models import Sequential,Input,Model
from keras.models import load_model #save and load models
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.preprocessing import LabelEncoder
import keras.backend as k

In [21]:
def income_level(row):
    if row['MedianIncome'] > 65000 :
        return 1
    else:
        return 0

df = pd.read_csv('uber_census_augmented_generated.csv', encoding = "ISO-8859-1", engine='python')
data = df
print(data.shape)
print(data.dtypes)
data['UserGroup'] = data.apply(lambda row: income_level(row), axis=1)
df1 = data
df1

(24380, 16)
hour                          float64
lat                           float64
long                          float64
MedianIncomeByPlaceofBirth    float64
MedianIncome                  float64
TotalPopulation               float64
AvgPopulation                 float64
Population5M                  float64
Population45M                 float64
Population4M                  float64
Population35M                 float64
Population3M                  float64
base_B02512                   float64
base_B02598                   float64
base_B02617                   float64
base_B02682                   float64
dtype: object


,hour,lat,long,MedianIncomeByPlaceofBirth,MedianIncome,TotalPopulation,AvgPopulation,Population5M,Population45M,Population4M,Population35M,Population3M,base_B02512,base_B02598,base_B02617,base_B02682,UserGroup
0,51.0,40.883700,-73.852900,35833.000000,7.596200e+04,3442.000000,3004.000000,1.860858e+06,1.628639e+06,1.351564e+06,1.090890e+06,8.919960e+05,0.0,0.0,1.0,0.0,1
1,77.0,40.860671,-73.793922,20138.373061,-3.439088e+07,4831.261780,4821.905285,1.123955e+06,3.552895e+06,3.146873e+05,1.117790e+06,8.042028e+05,0.0,0.0,1.0,0.0,0
2,35.0,40.661351,-73.918443,38706.025388,1.668560e+07,5997.650009,2189.453514,1.560699e+06,7.396952e+05,3.391752e+05,8.781857e+04,1.806951e+06,0.0,0.0,1.0,0.0,1
3,22.0,40.859819,-74.021596,42942.613228,-6.820204e+06,932.574501,2944.732906,2.286153e+06,1.685031e+06,1.567209e+06,7.179887e+05,7.218626e+05,0.0,0.0,1.0,0.0,0
4,54.0,40.838599,-74.160011,44336.958655,-1.217309e+07,3552.409578,4537.274043,1.988926e+06,2.594602e+06,4.712063e+05,1.327416e+06,5.708054e+05,0.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24375,8.0,40.969620,-73.765074,-3100.588832,2.013001e+07,10805.699263,9454.967927,1.742094e+06,8.359068e+05,2.241874e+06,1.001976e+06,1.533364e+06,0.0,0.0,1.0,0.0,1
24376,29.0,41.068968,-73.828891,32515.845294,2.546137e+07,3911.347486,4304.175988,2.883266e+06,7.491187e+05,2.875799e+06,9.958373e+05,6.094387e+05,0.0,0.0,1.0,0.0,1
24377,13.0,41.082909,-73.903289,39874.716654,1.282619e+07,6446.527125,8267.406045,2.836411e+06,2.062798e+06,7.063043e+05,1.414319e+06,1.582114e+06,0.0,0.0,1.0,0.0,1
24378,40.0,40.914238,-73.997844,35038.093905,-9.066141e+06,6843.655630,5638.818670,2.592380e+06,2.232494e+06,1.341607e+06,1.471795e+06,4.473418e+05,0.0,0.0,1.0,0.0,0


In [22]:
names_cloud = data.columns.tolist()
print(names_cloud)
X = np.array(data[names_cloud])
print(X.shape)

['hour', 'lat', 'long', 'MedianIncomeByPlaceofBirth', 'MedianIncome', 'TotalPopulation', 'AvgPopulation', 'Population5M', 'Population45M', 'Population4M', 'Population35M', 'Population3M', 'base_B02512', 'base_B02598', 'base_B02617', 'base_B02682', 'UserGroup']
(24380, 17)


In [23]:
y = (data['UserGroup']).values
print(pd.value_counts(pd.Series(y)))
data.drop('UserGroup',axis=1, inplace =True,)

0    13391
1    10989
dtype: int64


In [24]:
names_x = df1.columns.tolist()
print("Target Variable: UserGroup")
print("Predictors: "+str(names_x))
x = np.array(df1[names_x])
print("Number of data samples : {0:d}".format(x.shape[0]))
print("Number of Predictor Features : {0:d}".format(x.shape[1]))
df_ohc = df1[names_x]

Target Variable: UserGroup
Predictors: ['hour', 'lat', 'long', 'MedianIncomeByPlaceofBirth', 'MedianIncome', 'TotalPopulation', 'AvgPopulation', 'Population5M', 'Population45M', 'Population4M', 'Population35M', 'Population3M', 'base_B02512', 'base_B02598', 'base_B02617', 'base_B02682']
Number of data samples : 24380
Number of Predictor Features : 16


In [25]:
df_ohc.dtypes

hour                          float64
lat                           float64
long                          float64
MedianIncomeByPlaceofBirth    float64
MedianIncome                  float64
TotalPopulation               float64
AvgPopulation                 float64
Population5M                  float64
Population45M                 float64
Population4M                  float64
Population35M                 float64
Population3M                  float64
base_B02512                   float64
base_B02598                   float64
base_B02617                   float64
base_B02682                   float64
dtype: object

In [26]:
x = x.astype('float32')
x = x/255
batch_size = 64 #upto us
epochs = 200
lrate = 0.001
decay = 1e-7 
data_size = 2000
ns = 10 #number of shadow models for one data_size
nh = 8 #number of hidden layers
nout = 1
seed = 9
np.random.seed(seed)
sh = np.arange(x.shape[0])
np.random.shuffle(sh)
target_rep = np.zeros((1,x.shape[0]))
target_rep[0,:] = sh
print(sh)

[20628 12886  1155 ...   501 23166 20828]


In [27]:
from tensorflow.keras.optimizers import Adam
k.clear_session()
xtr_target = x[sh[:data_size]]
ytr_target = y[sh[:data_size]]
xts_target = x[sh[data_size:data_size*2]]
yts_target = y[sh[data_size:2*data_size]]
shadow_rep = np.zeros((20,x.shape[0]-2*data_size))
sh1 = sh[2*data_size:]
xtr_att = np.zeros((2*data_size*ns,1))
ytr_att = np.zeros((2*data_size*ns,1))
xtr_att_truelabels = np.zeros((2*data_size*ns,))
model_target = Sequential()
model_target.add(Dense(nh, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
model_target.add(Dense(1, activation='sigmoid', name = 'output'))
opt = Adam(lr=lrate, decay=decay) 
model_target.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
print(model_target.summary())
hist_target = model_target.fit(xtr_target, ytr_target,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_target, yts_target), shuffle=True, verbose=0)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 8)                 136       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [28]:
print('\n\nFor target model with training datasize = %d'%data_size)
print('Training accuracy = ', hist_target.history['accuracy'][-1])
print('Validation accuracy = ', hist_target.history['val_accuracy'][-1])
model_target_name = 'Uber_Census_target_'+str(data_size)+'.h5'
model_target.save(model_target_name)
ytemp_tr_target = model_target.predict(xtr_target)
ytemp_ts_target = model_target.predict(xts_target)
xts_att = np.vstack((ytemp_tr_target,ytemp_ts_target))
yts_att = np.zeros((2*data_size,1))
yts_att[data_size:2*data_size] = 1  
xts_att_truelabels = np.vstack((ytr_target,yts_target))
xts_att_dict = {'xts_att':xts_att,'yts_att':yts_att,'xts_att_truelabels':xts_att_truelabels}
fname = './att_test_data_'+str(data_size)
np.save(fname,xts_att_dict)
datafile = './data_ubercensus_target_'+str(data_size)
np.save(datafile,target_rep)



For target model with training datasize = 2000
Training accuracy =  0.9564999938011169
Validation accuracy =  0.9484999775886536


In [29]:
for i in np.arange(ns):
    np.random.shuffle(sh1)
    shadow_rep[i,:] = sh1
    xtr_shadow = x[sh1[:data_size]]
    ytr_shadow = y[sh1[:data_size]]
    xts_shadow = x[sh1[data_size:2*data_size]]
    yts_shadow = y[sh1[data_size:2*data_size]]
    model_shadow = Sequential()
    model_shadow.add(Dense(nh, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
    model_shadow.add(Dense(1, activation='sigmoid', name = 'output'))
    opt = Adam(learning_rate=lrate, decay=decay) 
    model_shadow.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    if i == 0:
        print("Shadow Model Summary")
        print(model_shadow.summary())
    hist_shadow = model_shadow.fit(xtr_shadow, ytr_shadow,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_shadow, yts_shadow), shuffle=True, verbose=0)
    print("Shadow model no: %d"%i)
    print('\n\nFor shadow model with training datasize = %d'%data_size)
    print('Training accuracy = ', hist_shadow.history['accuracy'][-1])
    print('Validation accuracy = ', hist_shadow.history['val_accuracy'][-1])
    ytemp11 = model_shadow.predict(xtr_shadow)
    ytemp22 = model_shadow.predict(xts_shadow)
    model_shadow_name = 'UBER_Census_shadow_'+str(data_size)+'_'+str(i)+'.h5'
    print(model_shadow_name)
    model_shadow.save(model_shadow_name)    

    print(ytemp11)
    print(ytemp22)

    xtr_att[i*2*data_size:(i+1)*2*data_size] = np.vstack((ytemp11,ytemp22))
    ytr_att[((i*2)+1)*data_size:(i+1)*2*data_size] = 1
    print('****')
    print(ytr_shadow)
    print(yts_shadow)
    print( np.hstack((ytr_shadow,yts_shadow)))
    xtr_att_truelabels[i*2*data_size:(i+1)*2*data_size] = np.hstack((ytr_shadow,yts_shadow))
datafile = './data_UBER_Census_shadow_'+str(data_size)
np.save(datafile,shadow_rep)
xtr_att_dict = {'xtr_att':xtr_att,'ytr_att':ytr_att,'xtr_att_truelabels':xtr_att_truelabels}
fname = './att_train_data_'+str(data_size)
np.save(fname,xtr_att_dict)

Shadow Model Summary
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 8)                 136       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________
None
Shadow model no: 0


For shadow model with training datasize = 2000
Training accuracy =  0.9610000252723694
Validation accuracy =  0.9639999866485596
UBER_Census_shadow_2000_0.h5
[[0.9990577 ]
 [0.9990577 ]
 [0.00336298]
 ...
 [0.9990577 ]
 [0.9990577 ]
 [0.00336298]]
[[0.00336298]
 [0.9990577 ]
 [0.3770604 ]
 ...
 [0.9990577 ]
 [0.9990577 ]
 [0.00336298]]
****
[1 1 0 ... 1 1 0]
[0 1 1 ... 1 1 0]
[1 1 0 ... 1 1 0]


KeyboardInterrupt: ignored

In [ ]:
model_attack = Sequential()
model_attack.add(Dense(nh, input_shape = (xtr_att.shape[1],), activation='sigmoid', name = 'hidden'))
model_attack.add(Dense(1, activation='sigmoid', name = 'output'))
opt = Adam(learning_rate=lrate, decay=decay) 
model_attack.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
print("Attack Model Summary")
print(model_attack.summary())
hist_attack = model_attack.fit(xtr_att, ytr_att,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_att, yts_att), shuffle=True, verbose=0)
print('\n\nFor attack model with training datasize = %d'%xtr_att.shape[0])
print('Training accuracy = ', hist_shadow.history['accuracy'][-1])
print('Validation accuracy = ', hist_shadow.history['val_accuracy'][-1])